In [ ]:
import streamlit as st
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import exchange_calendars as xcals
import numpy as np
import os
import time
import io
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# import general_function.general_func as gf

def formatea_precios_yahoo(bmks_rv, fecha):
    precios_bmks_yahoo_df = yf.download(bmks_rv, start=datetime(year=fecha.year - 1, month=1, day=1).strftime("%Y-%m-%d"))
    precios_bmks_yahoo_df = precios_bmks_yahoo_df.xs(key="Close", axis=1, level=0)
    precios_bmks_yahoo_df.reset_index(inplace=True)
    precios_bmks_yahoo_df.rename(columns={"Date":"Fecha", "^MXX":"IPC", "^SPESG":"SPESG_USD", "^SPGSCI":"SPGSCI_USD", "^GSPC":"S&P_USD", "^NDX":"NDX_USD"}, inplace=True)
    precios_bmks_yahoo_df["Fecha"] = pd.to_datetime(precios_bmks_yahoo_df["Fecha"])

    return precios_bmks_yahoo_df

def formatea_precios_spot():
    spot_df = pd.read_csv("./ArchivosRendimientos/Benchmarks/Historico_SPOT.csv")
    spot_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"Spot"}, inplace=True)
    spot_df = spot_df[["Fecha", "Spot"]]
    spot_df["Fecha"] = pd.to_datetime(spot_df["Fecha"])

    return spot_df

def formatea_precios_isimp():
    precios_bmks_isimp_df = pd.read_excel("./ArchivosPeergroups/Benchmarks/Historico_ISIMP.xlsx", skiprows=2, skipfooter=4)
    precios_bmks_isimp_df["Fecha"] = pd.to_datetime(precios_bmks_isimp_df["Fecha"], format="%d/%m/%Y")
    precios_bmks_isimp_df.rename(columns={"Índice":"ISIMP"}, inplace=True)
    precios_bmks_isimp_df = precios_bmks_isimp_df[["Fecha", "ISIMP"]]

    return precios_bmks_isimp_df

def formatea_precios_acwi():
    precios_bmks_acwi_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Historico_ACWI.csv")
    precios_bmks_acwi_df.rename(columns={"FECHA":"Fecha", "PRECIO SUCIO":"ACWI"}, inplace=True)
    precios_bmks_acwi_df["Fecha"] = pd.to_datetime(precios_bmks_acwi_df["Fecha"], format="%Y-%m-%d")
    precios_bmks_acwi_df = precios_bmks_acwi_df[["Fecha", "ACWI"]]

    return precios_bmks_acwi_df

def formatea_precios_bmks_valmer():
    precios_bmks_valmer_df = pd.read_csv("./ArchivosPeergroups/Benchmarks/Benchmarks_SP_Historico_MD.csv")
    precios_bmks_valmer_df.rename(columns={"FECHA":"Fecha"}, inplace=True)
    precios_bmks_valmer_df["Fecha"] = pd.to_datetime(precios_bmks_valmer_df["Fecha"], format="%Y%m%d")
    precios_bmks_valmer_df.drop(columns=[col for col in precios_bmks_valmer_df.columns if "Unnamed" in col], inplace=True)

    return precios_bmks_valmer_df

def formatea_precios_bmks(fecha):
    spot_df = formatea_precios_spot()
    precios_bmks_yahoo_df = formatea_precios_yahoo(bmks_rv, fecha)
    precios_bmks_df = pd.merge(precios_bmks_yahoo_df, spot_df, on="Fecha", how="outer")
    precios_bmks_df["SPESG"] = precios_bmks_df["SPESG_USD"] * precios_bmks_df["Spot"]
    precios_bmks_df["SPGSCI"] = precios_bmks_df["SPGSCI_USD"] * precios_bmks_df["Spot"]
    precios_bmks_df["S&P"] = precios_bmks_df["S&P_USD"] * precios_bmks_df["Spot"]
    precios_bmks_df["NDX"] = precios_bmks_df["NDX_USD"] * precios_bmks_df["Spot"]
    precios_bmks_df.drop(columns=["SPESG_USD", "SPGSCI_USD", "S&P_USD", "NDX_USD"], inplace=True)
    precios_bmks_isimp_df = formatea_precios_isimp()
    precios_bmks_df = pd.merge(precios_bmks_df, precios_bmks_isimp_df, on="Fecha", how="outer")
    precios_bmks_acwi_df = formatea_precios_acwi()
    precios_bmks_df = pd.merge(precios_bmks_df, precios_bmks_acwi_df, on="Fecha", how="outer")
    precios_bmks_valmer_df = formatea_precios_bmks_valmer()
    precios_bmks_df = pd.merge(precios_bmks_df, precios_bmks_valmer_df, on="Fecha", how="outer")
    precios_bmks_df.set_index("Fecha", inplace=True)
    precios_bmks_df.sort_index(inplace=True)

    return precios_bmks_df.ffill()

def infer_calendar(dates):
    """
    Infer a calendar as pandas DateOffset from a list of dates.
    Parameters
    ----------
    dates : array-like (1-dimensional) or pd.DatetimeIndex
        The dates you want to build a calendar from
    Returns
    -------
    calendar : pd.DateOffset (CustomBusinessDay)
    """
    dates = pd.DatetimeIndex(dates)

    traded_weekdays = []
    holidays = []

    days_of_the_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for day, day_str in enumerate(days_of_the_week):

        weekday_mask = (dates.dayofweek == day)

        # keep only days of the week that are present
        if not weekday_mask.any():
            continue
        traded_weekdays.append(day_str)

        # look for holidays
        used_weekdays = dates[weekday_mask].normalize()
        all_weekdays = pd.date_range(dates.min(), dates.max(),
                                     freq=CustomBusinessDay(weekmask=day_str)
                                     ).normalize()
        _holidays = all_weekdays.difference(used_weekdays)
        _holidays = [timestamp.date() for timestamp in _holidays]
        holidays.extend(_holidays)

    traded_weekdays = ' '.join(traded_weekdays)
    return CustomBusinessDay(weekmask=traded_weekdays, holidays=holidays)

def calcula_fechas_exactas_iniciales(fecha):
    if fecha.month == 1:
        year = fecha.year - 1
        month = 12
    else:
        year = fecha.year
        month = fecha.month - 1

    fechas_exactas_iniciales = {
        "MTD": datetime(year=year, month=month, day=calendar.monthrange(year, month)[1]),
        "YTD": datetime(year=fecha.year - 1, month=12, day=calendar.monthrange(year=fecha.year - 1, month=12)[1]),
        "12 Meses": fecha - timedelta(days=366 if calendar.isleap(year) else 365),
        "30D": fecha - timedelta(days=30),
        "90D": fecha - timedelta(days=90),
        "180D": fecha - timedelta(days=180)
    }

    return fechas_exactas_iniciales

def calcula_fecha_habil_proxima_anterior(fecha_exacta, fechas_bmv, bmv_offset):
    return (fecha_exacta - bmv_offset).to_pydatetime() if fecha_exacta not in fechas_bmv else fecha_exacta

def calcula_fecha_habil_proxima_posterior(fecha_exacta, fechas_bmv, bmv_offset):
    return (fecha_exacta + bmv_offset).to_pydatetime() if fecha_exacta not in fechas_bmv else fecha_exacta

def calcula_fechas_habiles_iniciales(fechas_exactas_iniciales, fechas_bmv, bmv_offset, tipo="Deuda"):
    if tipo == "Deuda":
        fechas_habiles_iniciales = {
            ventana:calcula_fecha_habil_proxima_anterior(fechas_exactas_iniciales[ventana], fechas_bmv, bmv_offset) for ventana in fechas_exactas_iniciales.keys()
        }
    
    else:
        fechas_habiles_iniciales = {
            ventana:calcula_fecha_habil_proxima_posterior(fechas_exactas_iniciales[ventana], fechas_bmv, bmv_offset) if ventana in ["MTD", "YTD"] else
            calcula_fecha_habil_proxima_anterior(fechas_exactas_iniciales[ventana], fechas_bmv, bmv_offset) for ventana in fechas_exactas_iniciales.keys()
        }

    return fechas_habiles_iniciales

def formatea_rendimientos_bmk(fecha, precios_bmks_df, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv, propiedades_fondos_df, bmv_offset, fondo2benchmark):
    rendimientos_bmks_df = pd.DataFrame()
    for ventana in fechas_habiles_iniciales_rv.keys():
        rendimientos_bmk_ventana = []
        for fondo in fondo2benchmark.keys():
            tipo_fondo = propiedades_fondos_df.loc[propiedades_fondos_df["Fondo"] == fondo, "Tipo"].item()
            fechas_habiles_iniciales = fechas_habiles_iniciales_rv if tipo_fondo == "RV" else fechas_habiles_iniciales_rf

            fecha_inicial = (fechas_habiles_iniciales[ventana] - bmv_offset).to_pydatetime()
            fecha_final = (fecha - bmv_offset).to_pydatetime()

            bmks = fondo2benchmark[fondo]["Benchmarks"]
            pesos = fondo2benchmark[fondo]["Pesos"]
            rendimiento_bmk = ((precios_bmks_df[bmks].loc[fecha_final]/precios_bmks_df[bmks].loc[fecha_inicial] - 1) * pesos).sum()

            if len(bmks) == 0:
                rendimiento_bmk = np.nan
            
            if tipo_fondo == "RF" or fondo in ["BALANCE", "DYNAMIC"]:
                rendimiento_bmk = rendimiento_bmk * 360/(fecha - fechas_habiles_iniciales[ventana]).days

            rendimientos_bmk_ventana.append(rendimiento_bmk)

        rendimientos_bmk_ventana_df = pd.DataFrame({ventana:rendimientos_bmk_ventana}, index=fondo2benchmark.keys())
        rendimientos_bmks_df = pd.concat([rendimientos_bmks_df, rendimientos_bmk_ventana_df], axis=1)

    return rendimientos_bmks_df

def formatea_rendimientos_fondos(fecha, precios_fondos_df, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv, propiedades_fondos_df, tipo_calculo="XF"):
    rendimientos_df = pd.DataFrame()
    for tipo in ["RF", "RV"]:
        fondos = propiedades_fondos_df.loc[propiedades_fondos_df["Tipo"] == tipo, "Fondo"].tolist()
        fechas_habiles_iniciales = fechas_habiles_iniciales_rf if tipo == "RF" else fechas_habiles_iniciales_rv
        temp_rendimientos_df = calcula_rendimientos_fondos(precios_fondos_df, fondos, fecha, fechas_habiles_iniciales)
        temp_propiedades_fondos_df = propiedades_fondos_df[~propiedades_fondos_df["Serie"].isin(["XF0", "XF"])].copy()
        if temp_rendimientos_df.index.isin(temp_propiedades_fondos_df["Fondo"]).any():
            temp_propiedades_fondos_df = temp_propiedades_fondos_df[
                (temp_propiedades_fondos_df["Tipo"] == tipo) &
                (~temp_propiedades_fondos_df["Fondo"].isin(["INCOME", "BALANCE", "DYNAMIC"]))
            ]
            temp_rendimientos_brutos_df = calcula_rendimientos_brutos(temp_rendimientos_df, temp_propiedades_fondos_df, 0.005, fecha, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv)
            temp_rendimientos_df.loc[temp_propiedades_fondos_df["Fondo"]] = temp_rendimientos_brutos_df.copy()

            temp_propiedades_fondos_df = temp_propiedades_fondos_df[
                (temp_propiedades_fondos_df["Tipo"] == tipo) &
                (temp_propiedades_fondos_df["Fondo"].isin(["INCOME", "BALANCE", "DYNAMIC"]))
            ]
            temp_rendimientos_brutos_df = calcula_rendimientos_brutos(temp_rendimientos_df, temp_propiedades_fondos_df, 0.005, fecha, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv)
            temp_rendimientos_df.loc[temp_propiedades_fondos_df["Fondo"]] = temp_rendimientos_brutos_df.copy()

        rendimientos_df = pd.concat([rendimientos_df, temp_rendimientos_df], axis=0)

    fondos_a_anualizar = propiedades_fondos_df.loc[propiedades_fondos_df["Tipo"] == "RF", "Fondo"].tolist() + ["BALANCE", "DYNAMIC"]
    temp_rendimientos_df = rendimientos_df.loc[fondos_a_anualizar].copy()
    temp_rendimientos_df = calcula_rendimientos_anualizados(temp_rendimientos_df, fecha, fechas_habiles_iniciales_rf)
    rendimientos_df.loc[fondos_a_anualizar] = temp_rendimientos_df.copy()

    return rendimientos_df

def formatea_columna_tabla_rendimientos_MiVector(col):
    temp_col = col.copy()
    if temp_col.dtype == "O":
        temp_col = temp_col.replace("N[/]D[\%]", "", regex=True)
        temp_col = temp_col.replace("%|,", "", regex=True)
        temp_col = temp_col.replace("", np.nan, regex=True)

        if temp_col.isna().sum() != len(temp_col):
            if not temp_col.str.contains(r"[A-Za-z]").any():
                temp_col = temp_col.astype(float)
                temp_col = temp_col/100 if col.name not in ["Precio", "(+)Títulos en Circulación"] else temp_col

    return temp_col

def descarga_rendimientos_MiVector(fechas):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=chrome_options)
    start_url = "https://www.vectoronline.com.mx/servicios/fondos/consultas/precios_rendimientos.php"
    driver.get(start_url)

    rendimientos_MiVector_fechas_df = pd.DataFrame()
    for fecha_dt in fechas:
        fecha_str = fecha_dt.strftime("%Y-%m-%d")
        time.sleep(3)

        driver.find_element(by=By.ID, value="dpFecha").clear()
        driver.find_element(by=By.ID, value="dpFecha").send_keys(fecha_str)
        driver.find_element(by=By.ID, value="btnMostrar").click()

        time.sleep(3)

        tables = pd.read_html(io.StringIO(driver.page_source))

        rendimientos_MiVector_df = []
        for i in range(len(tables)):
            if i % 2 != 0:
                col_names = tables[i - 1].columns
                temp_df = tables[i].rename(columns={i:col_names[i] for i in range(len(col_names))})
                temp_records = temp_df.to_dict(orient="records")
                rendimientos_MiVector_df.extend(temp_records)

        rendimientos_MiVector_df = pd.DataFrame.from_records(rendimientos_MiVector_df)
        rendimientos_MiVector_df["Fecha"] = fecha_str
        rendimientos_MiVector_fechas_df = pd.concat([rendimientos_MiVector_fechas_df, rendimientos_MiVector_df], axis=0, ignore_index=True)

    rendimientos_MiVector_fechas_df["Fecha"] = pd.to_datetime(rendimientos_MiVector_fechas_df["Fecha"], format="%Y-%m-%d")
    rendimientos_MiVector_fechas_df = rendimientos_MiVector_fechas_df.apply(lambda col: formatea_columna_tabla_rendimientos_MiVector(col), axis=0)

    driver.close()

    return rendimientos_MiVector_fechas_df

def calcula_rendimientos_fondos(precios_fondos_df, fondos, fecha, fechas_habiles_iniciales):
    rendimientos_df = pd.DataFrame()
    for ventana in fechas_habiles_iniciales.keys():
        fecha_ventana = fechas_habiles_iniciales[ventana]
        rendimientos_ventana_df = precios_fondos_df.loc[fecha, fondos]/precios_fondos_df.loc[fecha_ventana, fondos] - 1
        # if "VECTRF" in fondos:
        #     st.write(ventana)
        #     st.write(rendimientos_ventana_df)
        rendimientos_ventana_df.name = ventana
        rendimientos_ventana_df = rendimientos_ventana_df.to_frame()
        rendimientos_df = pd.concat([rendimientos_df, rendimientos_ventana_df], axis=1)

    return rendimientos_df

def calcula_rendimientos_anualizados(rendimientos_df, fecha, fechas_habiles_iniciales):
    rendimientos_anualizados_df = rendimientos_df.copy()
    for ventana in rendimientos_anualizados_df:
        dias_transcurridos = (fecha - fechas_habiles_iniciales[ventana]).days
        rendimientos_anualizados_df[ventana] = rendimientos_anualizados_df[ventana] * 360 / dias_transcurridos

    return rendimientos_anualizados_df

def calcula_rendimientos_brutos(rendimientos_df, propiedades_fondos_df, impuesto, fecha, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv, tipo="Deuda"):
    temp_df = propiedades_fondos_df.copy()
    temp_df[["Comision", "Factor RF"]] /= 100

    # dias_transcurridos_ventanas_df = pd.DataFrame([
    #         {ventana:1 if tipo != "Estrategicos" else (fecha - fechas_habiles_iniciales_rf[ventana]).days for ventana in fechas_habiles_iniciales_rf}|{"Tipo":"RF"},
    #         {ventana:1 if tipo != "Estrategicos" else (fecha - fechas_habiles_iniciales_rv[ventana]).days for ventana in fechas_habiles_iniciales_rv}|{"Tipo":"RV"}
    # ])

    dias_transcurridos_ventanas_df = pd.DataFrame([
            {ventana:(fecha - fechas_habiles_iniciales_rf[ventana]).days for ventana in fechas_habiles_iniciales_rf}|{"Tipo":"RF"},
            {ventana:(fecha - fechas_habiles_iniciales_rv[ventana]).days for ventana in fechas_habiles_iniciales_rv}|{"Tipo":"RV"}
    ])

    temp_df = pd.merge(temp_df, dias_transcurridos_ventanas_df, on="Tipo")
    temp_df.set_index("Fondo", inplace=True)
    temp_df.index.name = None

    comisiones_srs = temp_df["Comision"]
    # ajustes_comision_srs = (comisiones_srs * 1.16) / (1 if tipo != "Estrategicos" else 360)
    ajustes_comision_srs = (comisiones_srs * 1.16) / 360
    ajustes_comision_df = temp_df[list(fechas_habiles_iniciales_rf.keys())].multiply(ajustes_comision_srs, axis=0)

    factores_rf_srs = temp_df["Factor RF"]
    # ajustes_impuesto_srs = (factores_rf_srs * impuesto) / (1 if tipo != "Estrategicos" else 360)
    ajustes_impuesto_srs = (factores_rf_srs * impuesto) / 360
    ajustes_impuesto_df = temp_df[list(fechas_habiles_iniciales_rf.keys())].multiply(ajustes_impuesto_srs, axis=0)

    rendimientos_brutos_df = rendimientos_df.loc[temp_df.index.tolist()].copy()
    if tipo != "Estrategicos":
        rendimientos_brutos_df = rendimientos_brutos_df + ajustes_comision_df + ajustes_impuesto_df
    else:
        rendimientos_brutos_df = (1 + rendimientos_brutos_df)/(1 - ajustes_comision_df) + ajustes_impuesto_df - 1

    return rendimientos_brutos_df

# def calcula_rendimientos_brutos(rendimientos_df, propiedades_fondos_df, impuesto, fecha, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv):
#     temp_df = propiedades_fondos_df.copy()
#     temp_df[["Comision", "Factor RF"]] /= 100

#     dias_transcurridos_ventanas_df = pd.DataFrame([
#             {ventana:(fecha - fechas_habiles_iniciales_rf[ventana]).days for ventana in fechas_habiles_iniciales_rf}|{"Tipo":"RF"},
#             {ventana:(fecha - fechas_habiles_iniciales_rv[ventana]).days for ventana in fechas_habiles_iniciales_rv}|{"Tipo":"RV"}
#     ])

#     temp_df = pd.merge(temp_df, dias_transcurridos_ventanas_df, on="Tipo")
#     temp_df.set_index("Fondo", inplace=True)
#     temp_df.index.name = None

#     comisiones_srs = temp_df["Comision"]
#     ajustes_comision_srs = (comisiones_srs * 1.16)/ 360
#     ajustes_comision_df = temp_df[list(fechas_habiles_iniciales_rf.keys())].multiply(ajustes_comision_srs, axis=0)

#     factores_rf_srs = temp_df["Factor RF"]
#     ajustes_impuesto_srs = (factores_rf_srs * impuesto)/360
#     ajustes_impuesto_df = temp_df[list(fechas_habiles_iniciales_rf.keys())].multiply(ajustes_impuesto_srs, axis=0)

#     rendimientos_brutos_df = rendimientos_df.loc[temp_df.index.tolist()].copy()
#     rendimientos_brutos_df = (1 + rendimientos_brutos_df)/(1 - ajustes_comision_df) + ajustes_impuesto_df - 1

#     return rendimientos_brutos_df


def grafico_diferencias_rendimiento(rendimientos_df, periodo="MTD", titulo_adicional=""):
    """
    Genera un gráfico de barras horizontales que muestra las diferencias entre los rendimientos
    de los fondos y sus benchmarks para un período específico.
    
    Parámetros:
    rendimientos_df (pandas.DataFrame): DataFrame con los fondos como índice y columnas para rendimientos
    periodo (str): Período a visualizar ("MTD", "YTD", "12 Meses", "30D", "90D", "180D")
    titulo_adicional (str): Texto adicional para el título del gráfico
    
    Retorna:
    fig (plotly.graph_objects.Figure): Figura de Plotly con el gráfico
    """
    # Definir la paleta de colores personalizada
    custom_colors = ['#EC5E2A', '#FF8F66', '#1A3A6C', '#2E5095', '#4268B1', '#5680CE', '#6A98EB']
    
    # Reemplazar valores NaN con 0
    rendimientos_df = rendimientos_df.fillna(0)
    
    # Determinar columnas para fondos y benchmarks
    col_benchmark = f"BMK_{periodo}"
    
    # Verificar si las columnas existen
    if periodo not in rendimientos_df.columns or col_benchmark not in rendimientos_df.columns:
        raise ValueError(f"No se encontraron las columnas {periodo} o {col_benchmark} en el DataFrame")
    
    # Crear una copia del DataFrame para no modificar el original
    df_trabajo = rendimientos_df.copy()
    
    # Caso especial para DYNAMIC: usar 12.0 como benchmark para todas las columnas
    if 'DYNAMIC' in df_trabajo.index:
        for col in df_trabajo.columns:
            if col.startswith('BMK_'):
                df_trabajo.loc['DYNAMIC', col] = 12.0
    
    # Calcular diferencias
    df_trabajo['Diferencia'] = df_trabajo[periodo] - df_trabajo[col_benchmark]
    
    # Dividir en overweight y underweight
    df_trabajo['Underweight'] = df_trabajo['Diferencia'].copy()
    df_trabajo['Overweight'] = df_trabajo['Diferencia'].copy()
    df_trabajo.loc[df_trabajo['Underweight'] > 0, 'Underweight'] = 0
    df_trabajo.loc[df_trabajo['Overweight'] < 0, 'Overweight'] = 0
    
    # Ordenar por diferencia
    df_trabajo = df_trabajo.sort_values(by='Diferencia')
    
    # Crear el gráfico de barras horizontales apiladas en Plotly
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        y=df_trabajo.index,  # Usar el índice como nombres de fondos
        x=df_trabajo['Underweight'],
        orientation='h',
        marker=dict(
            color=custom_colors[2],
            line=dict(color='white', width=1)  # Agregar borde blanco
        ),
        name='Underperformance'
    ))
    
    fig.add_trace(go.Bar(
        y=df_trabajo.index,  # Usar el índice como nombres de fondos
        x=df_trabajo['Overweight'],
        orientation='h',
        marker=dict(
            color=custom_colors[0],
            line=dict(color='white', width=1)  # Agregar borde blanco
        ),
        name='Outperformance'
    ))
    
    # Añadir una línea vertical en el eje x=0
    fig.add_shape(
        type="line",
        x0=0, x1=0, y0=-0.5, y1=len(df_trabajo)-0.5,
        line=dict(color="white", width=0.8)
    )
    
    # Añadir anotaciones para mostrar los valores
    annotations = []
    for i, (idx, row) in enumerate(df_trabajo.iterrows()):
        if row['Diferencia'] < 0:
            annotations.append(dict(
                x=row['Underweight'], 
                y=idx,  # Usar el índice como nombre del fondo
                text=f"{abs(row['Underweight']):.2f}", 
                xanchor='right', 
                showarrow=False, 
                font=dict(color='white')
            ))
        else:
            annotations.append(dict(
                x=row['Overweight'], 
                y=idx,  # Usar el índice como nombre del fondo
                text=f"{row['Overweight']:.2f}", 
                xanchor='left', 
                showarrow=False, 
                font=dict(color='white')
            ))
    
    # Ajustar altura del gráfico según la cantidad de fondos
    height = max(500, 100 + len(df_trabajo) * 40)  # 40px por fondo + margen
    
    # Configurar etiquetas y título
    fig.update_layout(
        annotations=annotations,
        xaxis_title='Diferencia de Rendimiento %',
        title=f'Diferencias de Rendimiento entre Fondo y Benchmark ({periodo}){" - " + titulo_adicional if titulo_adicional else ""}',
        barmode='stack',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=height,
        width=1000,
        margin=dict(l=200),
        plot_bgcolor='#0C2653',  # Color de fondo del gráfico
        paper_bgcolor='#0C2653',  # Color de fondo del papel
        font=dict(color='#FFFFFF')  # Color del texto
    )
    
    # Asegurar que todas las etiquetas se muestren completamente
    fig.update_yaxes(tickfont=dict(size=14))
    fig.update_xaxes(gridcolor='#44475A')
    fig.update_yaxes(gridcolor='#44475A')
    
    return fig

def visualiza_precios_indizados_fondo_bmk(precios_indizados_fondo_bmks_df):
    '''
    Description:
        Formatea la gráfica de líneas Fecha VS Precio Indizado de un fondo y
        su benchmark.
    Input:
        - precios_indizados_fondo_bmks_df: pandas.DataFrame donde el índice
        sea la fecha y las columnas sean los rendimientos del fondo y su benchmark
        respecto a una fecha inicial fija.
    Output:
        - fig: plotly.Figure con la gráfica formateada
    '''
    precios_indizados_fondo_bmks_df_vis = precios_indizados_fondo_bmks_df.copy()
    precios_indizados_fondo_bmks_df_vis *= 100
    precios_indizados_fondo_bmks_df_vis.reset_index(inplace=True)
    precios_indizados_fondo_bmks_df_vis = precios_indizados_fondo_bmks_df_vis.melt(id_vars="Fecha", value_name="Rendimiento (%)", var_name="Portafolio")

    # st.write(precios_indizados_fondo_bmks_df_vis)

    fig = go.Figure()
    temp_fig = px.line(precios_indizados_fondo_bmks_df_vis, x="Fecha", y="Rendimiento (%)", color="Portafolio")
    for data in temp_fig["data"]:
        data["line"]["color"] = "white" if data["legendgroup"] == "BMK" else "#EC5E2A"
        data["line"]["width"] = 2
        
        fig.add_trace(go.Scatter(data))

    fig.update_xaxes(showgrid=True, linecolor='white', tickangle=90, tickfont=dict(size=15, color="white"), linewidth=4, mirror=True, title=None)
    fig.update_yaxes(showgrid=True, gridcolor='#44475A', linecolor='white', tickfont=dict(size=17, color="white"), title=dict(text='Rendimiento (%)', font=dict(size=30, color='white')),  linewidth=4, mirror=True)
    fig.update_layout(template="none", margin=dict(l=120, t=20, r=20, b=80), paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0.3)", legend=dict(font=dict(color="white", size=20)))

    return fig

In [100]:
fondo2benchmark = {
    "VECTUSA":{
        "Benchmarks":[
            "SPESG"
        ],
        "Pesos":[1.0]
    },
    "VECTCOB":{
        "Benchmarks":[
            "Spot"
        ],
        "Pesos":[1.0]
    },
    "VECTCOR":{
        "Benchmarks":[
            "ISIMP"
        ],
        "Pesos":[1.0]
    },
    "VECTUSD":{
        "Benchmarks":[
            "Spot"
        ],
        "Pesos":[1.0]
    },
    "VECTFI":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 7+ Day Bond Index",
            "S&P/BMV Sovereign MBONOS 1-3 Year Bond Index"
        ],
        "Pesos":[0.5, 0.5]
    },
    "VECTIND":{
        "Benchmarks":[
            "IPC"
        ],
        "Pesos":[1.0]
    },
    "VECTMD":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 28 Day Bond Index"
        ],
        "Pesos":[1.0]
    },
    "DYNAMIC":{
        "Benchmarks":[],
        "Pesos":[]
    },
    "VECTPA":{
        "Benchmarks":[
            "IPC"
        ],
        "Pesos":[1.0]
    },
    "VECTPRE":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 28 Day Bond Index"
        ],
        "Pesos":[1.0]
    },
    "VECTPZO":{
        "Benchmarks":[
            "S&P/BMV Sovereign MBONOS 3-5 Year Bond Index"
        ],
        "Pesos":[1.0]
    },
    "VECTRF":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 28 Day Bond Index"
        ],
        "Pesos":[1.0]
    },
    "VECTSIC":{
        "Benchmarks":[
            "ACWI"
        ],
        "Pesos":[1.0]
    },
    "INCOME":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 28 Day Bond Index", 
            "S&P/BMV Sovereign CETES 7+ Day Bond Index", 
            "S&P/BMV Sovereign MBONOS 1-3 Year Bond Index"
        ],
        "Pesos":[0.4, 0.3, 0.3]
    },
    "EQUITY":{
        "Benchmarks":[
            "ACWI"
        ],
        "Pesos":[1.0]
    },
    "BALANCE":{
        "Benchmarks":[
            "S&P/BMV Sovereign CETES 28 Day Bond Index", 
            "IPC", 
            "ACWI"
        ],
        "Pesos":[0.7, 0.15, 0.15]
    },
    "VECTTR":{
        "Benchmarks":[
            "S&P/BMV Sovereign UDIBONOS 1-3 Year Bond Index",
            "S&P/BMV Sovereign UDIBONOS 5-10 Year Bond Index"
        ],
        "Pesos":[0.4, 0.6]
    },
    "VECTMIX":{
        "Benchmarks":[
            "SPESG", "IPC"
        ],
        "Pesos":[0.5, 0.5]
    },
    "COMMODQ":{
        "Benchmarks":[
            "SPGSCI"
        ],
        "Pesos":[1.0]
    },
    "MXRATES":{
        "Benchmarks":[],
        "Pesos":[]
    },
    "NEXTGEN":{
        "Benchmarks":[
            "NDX"
        ],
        "Pesos":[1.0]
    }
}

In [101]:
bmks_rv = ["^MXX", "^SPESG", "^SPGSCI", "^GSPC", "^NDX"]

propiedades_fondos_path = "./ArchivosRendimientos/PropiedadesFondos"
propiedades_fondos_filename = os.listdir(propiedades_fondos_path)[0]
propiedades_fondos_df = pd.read_excel(os.path.join(propiedades_fondos_path, propiedades_fondos_filename))

precios_fondos_valmer_path = "./ArchivosPeergroups/PreciosFondosValmer"
precios_fondos_valmer_filename = os.listdir(precios_fondos_valmer_path)[0]
precios_fondos_valmer_df = pd.read_csv(os.path.join(precios_fondos_valmer_path, precios_fondos_valmer_filename))
precios_fondos_valmer_df = precios_fondos_valmer_df[["FECHA", "EMISORA", "SERIE", "PRECIO SUCIO"]]
precios_fondos_valmer_df.rename(columns={"FECHA":"Fecha", "EMISORA":"Fondo", "SERIE":"Serie", "PRECIO SUCIO":"Precio"}, inplace=True)
precios_fondos_valmer_df["Fecha"] = pd.to_datetime(precios_fondos_valmer_df["Fecha"], format="%Y-%m-%d")

In [103]:
tipo_calculo = "CONFERENCE"

propiedades_fondos_df = propiedades_fondos_df[propiedades_fondos_df["TipoCalculo"] == tipo_calculo]

precios_fondos_df = precios_fondos_valmer_df.copy()
precios_fondos_df = precios_fondos_df[
    pd.Series(list(zip(precios_fondos_df["Fondo"], precios_fondos_df["Serie"]))).isin(list(zip(propiedades_fondos_df["Fondo"], propiedades_fondos_df["Serie"])))
].reset_index(drop=True)

fechas = [datetime.today()]
temp_precios_fondos_df = descarga_rendimientos_MiVector(fechas)
temp_precios_fondos_df = temp_precios_fondos_df[
    pd.Series(list(zip(temp_precios_fondos_df["Fondo"], temp_precios_fondos_df["Serie"]))).isin(list(zip(propiedades_fondos_df["Fondo"], propiedades_fondos_df["Serie"])))
].reset_index(drop=True)
temp_precios_fondos_df = temp_precios_fondos_df[["Fecha", "Fondo", "Serie", "Precio"]]

precios_fondos_df = pd.concat([precios_fondos_df, temp_precios_fondos_df], axis=0, ignore_index=True)
precios_fondos_df = precios_fondos_df[["Fecha", "Fondo", "Precio"]].pivot(index="Fecha", columns="Fondo")
precios_fondos_df = precios_fondos_df.droplevel(level=0, axis=1)
precios_fondos_df.columns.name = None
precios_fondos_df.tail()

,BALANCE,COMMODQ,DYNAMIC,EQUITY,INCOME,MXRATES,NEXTGEN,VECTCOB,VECTCOR,VECTFI,...,VECTMD,VECTMIX,VECTPA,VECTPRE,VECTPZO,VECTRF,VECTSIC,VECTTR,VECTUSA,VECTUSD
Fecha,,,,,,,,,,,,,,,,,,,,,
2025-05-21,1.869732,1.348857,19.043272,1.760560,0.808205,1.461951,1.610556,8.600986,2.396897,30.327244,...,25.707277,1.868052,3.049174,3.097135,21.100471,2.633011,1.986325,1.599397,2.345904,2.309652
2025-05-22,1.867776,1.347900,19.046491,1.745934,0.806970,1.462383,1.595661,8.643626,2.395083,30.276220,...,25.710227,1.860822,3.064389,3.097608,21.005108,2.633540,1.969773,1.594194,2.318747,2.320386
2025-05-23,1.864735,1.332863,19.049856,1.739792,0.807880,1.463598,1.588351,8.616855,2.397517,30.323155,...,25.717079,1.847882,3.031381,3.098334,21.075420,2.634069,1.963010,1.599314,2.310432,2.314818
2025-05-26,1.864146,1.335517,19.059055,1.726983,0.807912,1.463142,1.574977,8.584002,2.397902,30.314689,...,25.731179,1.843791,3.058110,3.100099,21.046173,2.635660,1.948416,1.599024,2.283511,2.306347
2025-05-28,1.872422,1.336189,19.103018,1.759307,0.808976,1.464104,1.603649,8.596347,2.400615,30.371255,...,25.743279,1.868943,3.072413,3.101471,21.125693,2.636718,1.985266,1.599065,2.332514,2.310114


In [112]:
fecha = datetime(year=2025, month=5, day=2)

xmex = xcals.get_calendar("XMEX")
fechas_bmv = sorted(xmex.sessions_in_range(
    start=(datetime.today() - relativedelta(years=20) + timedelta(days=100)).strftime("%Y-%m-%d"), 
    end=datetime.today().strftime("%Y-%m-%d")
).to_pydatetime())
fechas_bmv.remove(datetime(2024, 10, 1, 0, 0))

bmv_offset = infer_calendar(fechas_bmv)

fechas_exactas_iniciales_rf = calcula_fechas_exactas_iniciales(fecha)
fechas_habiles_iniciales_rf = calcula_fechas_habiles_iniciales(fechas_exactas_iniciales_rf, fechas_bmv, bmv_offset, tipo="Deuda")

fechas_exactas_iniciales_rv = calcula_fechas_exactas_iniciales((fecha - bmv_offset).to_pydatetime())
fechas_exactas_iniciales_rv = {
    ventana:(fechas_exactas_iniciales_rv[ventana] + bmv_offset).to_pydatetime() if ventana in ["MTD", "YTD"] else
    fechas_exactas_iniciales_rv[ventana] for ventana in fechas_exactas_iniciales_rv.keys()
}
fechas_habiles_iniciales_rv = calcula_fechas_habiles_iniciales(fechas_exactas_iniciales_rv, fechas_bmv, bmv_offset, tipo="RV")

In [113]:
precios_bmks_df = formatea_precios_bmks(fecha)
precios_bmks_df.tail()

[*********************100%***********************]  4 of 5 completed

5 Failed downloads:
['^MXX']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%5EMXX?period1=1704088800&period2=1748456771&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=Too+Many+Requests%0D%0A (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1006)')))"))
[*********************100%***********************]  4 of 5 completed['^SPGSCI']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%5ESPGSCI?period1=1704085200&period2=1748456771&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=Too+Many+Requests%0D%0A (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED]

,IPC,Spot,SPESG,SPGSCI,S&P,NDX,ISIMP,ACWI,S&P/BMV Government MBONOS 5-10 Year Bond Index,S&P/BMV Government MBONOS 1-5 Year Bond Index,...,S&P/BMV Sovereign UDIBONOS & CBICS 1-3 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 3-5 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 5-10 Year Bond Index,S&P/BMV Sovereign UDIBONOS & CBICS 10+ Year Bond Index,S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (MXN),S&P/BMV Sovereign BONDESF Bond Index (MXN),S&P/BMV Sovereign BONDES Bond Index (MXN),S&P/BMV Sovereign International UMS 5-10 Year Target Maturity 30% Capped Bond Index (USD),S&P/BMV Green Social & Sustainable Target Duration Bond Index (MXN),S&P/BMV Overnight TIIE Funding Rate Index
Fecha,,,,,,,,,,,,,,,,,,,,,
2025-05-20,NaN,19.2757,NaN,NaN,NaN,NaN,296.472178,2395.776753,508.022636,448.835505,...,158.885026,147.706115,144.826483,130.472068,128.878291,142.720519,142.705294,124.253123,163.063360,343.077132
2025-05-21,NaN,19.3716,NaN,NaN,NaN,NaN,296.536399,2377.863900,506.478838,448.679038,...,158.832473,147.521926,144.545380,129.977532,129.004960,142.755409,142.740095,124.021172,162.948400,343.158231
2025-05-22,NaN,19.3121,NaN,NaN,NaN,NaN,296.564232,2369.208428,507.726204,449.176778,...,159.202292,147.882545,144.948481,130.726308,128.736462,142.790257,142.774902,123.744473,163.095217,343.239541
2025-05-23,NaN,19.2398,NaN,NaN,NaN,NaN,296.635736,2353.027540,506.859967,449.014256,...,159.313639,147.966421,145.019515,130.475858,128.578911,142.825457,142.809982,123.900173,163.037308,343.320869
2025-05-26,NaN,19.2359,NaN,NaN,NaN,NaN,296.782892,2352.550570,506.855978,449.220132,...,159.424247,148.070927,145.125445,130.376285,128.262907,142.927770,142.912290,123.955802,163.136573,343.564627


In [114]:
rendimientos_bmks_df = formatea_rendimientos_bmk(fecha, precios_bmks_df, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv, propiedades_fondos_df, bmv_offset, fondo2benchmark)
rendimientos_bmks_df.reset_index(inplace=True)
rendimientos_bmks_df.rename(columns={"index":"Fondo"}|{col:f"BMK_{col}" for col in rendimientos_bmks_df.columns if col != "index"}, inplace=True)
rendimientos_bmks_df

,Fondo,BMK_MTD,BMK_YTD,BMK_12 Meses,BMK_30D,BMK_90D,BMK_180D
0,VECTUSA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,VECTCOB,-0.041387,-0.060782,0.000000,-0.039345,-0.044963,-0.019854
2,VECTCOR,0.047910,0.096157,0.100417,0.079235,0.092350,0.097242
3,VECTUSD,-0.041387,-0.060782,0.000000,-0.039345,-0.044963,-0.019854
4,VECTFI,0.097629,0.143627,0.131066,0.115141,0.138078,0.135954
5,VECTIND,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,VECTMD,0.045131,0.094308,0.105983,0.084182,0.092158,0.096699
7,DYNAMIC,NaN,NaN,NaN,NaN,NaN,NaN
8,VECTPA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,VECTPRE,0.045131,0.094308,0.105983,0.084182,0.092158,0.096699


In [115]:
rendimientos_fondos_df = formatea_rendimientos_fondos(fecha, precios_fondos_df, fechas_habiles_iniciales_rf, fechas_habiles_iniciales_rv, propiedades_fondos_df, tipo_calculo)
rendimientos_fondos_df.reset_index(inplace=True)
rendimientos_fondos_df.rename(columns={"index":"Fondo"}, inplace=True)
rendimientos_fondos_df

,Fondo,MTD,YTD,12 Meses,30D,90D,180D
0,VECTRF,0.091637,0.097087,0.108259,0.091060,0.095031,0.100484
1,VECTPRE,0.102089,0.103656,0.112406,0.097618,0.102753,0.105331
2,VECTMD,0.105506,0.107745,0.113526,0.101886,0.107477,0.108388
3,VECTFI,0.062827,0.191538,0.151325,0.198048,0.184648,0.139491
4,VECTCOR,0.047970,0.149005,0.129259,0.100381,0.152972,0.129904
5,VECTTR,0.165271,0.161177,0.135366,0.053581,0.158305,0.111843
6,VECTPZO,-0.077723,0.249672,0.139544,0.199565,0.227808,0.147066
7,INCOME,-0.013417,0.206380,0.100122,0.183723,0.196495,0.126979
8,VECTCOB,-0.041334,-0.061212,0.150036,-0.039292,-0.045311,-0.020273
9,VECTIND,0.073813,0.133210,0.000457,0.060073,0.089938,0.116622


In [ ]:
rendimientos_df = pd.merge(rendimientos_fondos_df, rendimientos_bmks_df, on="Fondo")
rendimientos_df = rendimientos_df[["Fondo"] + sum([[col, f"BMK_{col}"] for col in fechas_exactas_iniciales_rf.keys()], [])]
rendimientos_df.set_index("Fondo", inplace=True)
rendimientos_df *= 100
rendimientos_df = rendimientos_df.round(decimals=2)
rendimientos_df

,MTD,BMK_MTD,YTD,BMK_YTD,12 Meses,BMK_12 Meses,30D,BMK_30D,90D,BMK_90D,180D,BMK_180D
Fondo,,,,,,,,,,,,
VECTRF,9.11,9.39,9.74,9.51,10.93,10.60,9.11,9.39,9.54,9.34,10.10,9.72
VECTPRE,9.83,9.39,10.39,9.51,11.34,10.60,9.83,9.39,10.32,9.34,10.58,9.72
VECTMD,10.21,9.39,10.80,9.51,11.45,10.60,10.21,9.39,10.81,9.34,10.88,9.72
VECTFI,23.27,12.80,19.43,14.43,15.30,13.05,23.27,12.80,19.06,14.20,14.10,13.63
VECTCOR,12.89,8.95,15.11,9.69,13.07,10.03,12.89,8.95,15.65,9.28,13.13,9.78
VECTTR,9.39,4.76,16.15,16.34,13.90,12.77,9.39,4.76,16.72,16.57,11.17,12.32
VECTPZO,28.00,15.58,25.60,23.11,14.17,15.02,28.00,15.58,24.22,22.94,15.03,18.31
INCOME,25.23,11.44,22.50,12.46,11.58,12.07,25.23,11.44,21.89,12.26,14.35,12.07
VECTCOB,-4.32,-4.32,-6.32,-6.26,14.03,0.00,-3.59,-3.59,-4.74,-4.70,-2.99,-2.95
